In [37]:
!pip install --quiet gradio langchain langchain_openai langchain-community
!pip install --quiet tavily-python

In [38]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-YaLgqvYmqPyynwwPQ6uyT3BlbkFJxRmMVVNUtIwauJBEwDIN"
os.environ["TAVILY_API_KEY"] = "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"

In [39]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [40]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [41]:
pip install wikipedia

In [42]:
import os
from langchain_community.utilities import BingSearchAPIWrapper
os.environ["BING_SUBSCRIPTION_KEY"] = "62128ffb990a4bb6bf48d09a735e4429"
os.environ["BING_SEARCH_URL"] = "https://api.bing.microsoft.com/v7.0/search"

In [54]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, Tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)



In [76]:
#tools = [TavilySearchResults(max_results=1), WikipediaQueryRun(api_wrapper=api_wrapper), BingSearchAPIWrapper().run]
tools = [
    Tool(
        name = "Bing search",
        func=BingSearchAPIWrapper().run,
        description="useful for when you need to answer questions about current events"
        )]

''' tools = [Tool (
    name = "Wikipedia",
    func=WikipediaQueryRun(api_wrapper=api_wrapper).run,
    description="useful for when you need to answer questions about current events"
)]

tools = [Tool (
    name = "Tavily",
    func=TavilySearchResults(max_results=1),
    description="useful for when you need to answer questions about current events"
)]
'''

' tools = [Tool (\n    name = "Wikipedia",\n    func=WikipediaQueryRun(api_wrapper=api_wrapper).run,\n    description="useful for when you need to answer questions about current events"\n)]\n\ntools = [Tool (\n    name = "Tavily",\n    func=TavilySearchResults(max_results=1),\n    description="useful for when you need to answer questions about current events"\n)]\n'

In [77]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the tavily_search_results_json tool for information.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [78]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [80]:
tools[0].function.name

AttributeError: 'Tool' object has no attribute 'function'

In [79]:
input_text = "What is the capital of France?"
dictionary = agent_executor.invoke({"input":input_text})
dictionary["output"]



> Entering new AgentExecutor chain...


BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'tools[0].function.name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'tools[0].function.name', 'code': 'invalid_value'}}

In [ ]:
import gradio as gr
def invoke_agent (input_text):
    dictionary = agent_executor.invoke({"input":input_text})
    return dictionary["output"]

In [ ]:
iface = gr.Interface(
    fn=invoke_agent,
    inputs=gr.Textbox(label="Ask a question"),
    outputs=gr.Textbox(label = "Answer"))
iface.launch()